In [ ]:
!pip install keras-ocr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.3 MB/s eta 0:00:00


In [ ]:

import math
import keras_ocr
import numpy as np
import matplotlib.pyplot as plt

path_img = 's.png'

# Create the pipeline for Keras OCR
pipeline = keras_ocr.pipeline.Pipeline()

# Recognize text using Keras OCR
keras_ocr_results = pipeline.recognize([path_img])
print(keras_ocr_results )

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 3s 3s/step
[[('s', array([[ 24., 127.],
       [ 43., 127.],
       [ 43., 148.],
       [ 24., 148.]], dtype=float32)), ('uf', array([[213.45627, 165.16711],
       [250.80109, 157.30505],
       [255.70294, 180.58887],
       [218.35812, 188.45093]], dtype=float32)), ('6', array([[198., 162.],
       [216., 162.],
       [216., 181.],
       [198., 181.]], dtype=float32)), ('e', array([[ 55., 248.],
       [ 94., 248.],
       [ 94., 278.],
       [ 55., 278.]], dtype=float32)), ('3', array([[211., 293.],
       [227., 293.],
       [227., 313.],
       [211., 313.]], dtype=float32)), ('s', array([[224., 293.],
       [252., 293.],
       [252., 313.],
       [224., 313.]], dtype=float32))]]


In [ ]:
import math
import keras_ocr
import numpy as np

path_img = 'mm.jpg'

# Create the pipeline for Keras OCR
pipeline = keras_ocr.pipeline.Pipeline()

# Recognize text using Keras OCR
keras_ocr_results = pipeline.recognize([path_img])
print(keras_ocr_results)

keras_ocr_results_list = []
list_bbox_forCombinedBbox = []

for prediction_group in keras_ocr_results:
    for i, prediction in enumerate(prediction_group):
        # Extract bounding box coordinates from prediction
        keras_x_min, keras_y_min = prediction[1][0]  # Assuming bbox is a list containing coordinates

        # Check if the prediction contains both digits and non-digits
        if any(char.isdigit() for char in prediction[0]) and any(not char.isdigit() for char in prediction[0]):
            # If it contains a mix, do not combine and add it directly
            keras_ocr_results_list.append(prediction)
            continue

        if i > 0:
            previous_prediction = prediction_group[i - 1]
            prev_x_min, prev_y_min = previous_prediction[1][0]

        if i < len(prediction_group) - 1:
            next_prediction = prediction_group[i + 1]
            next_x_min, next_y_min = next_prediction[1][0]

        # Check conditions for combining predictions keras_y_min - next_y_min < 10
        if (i > 0 and
            keras_x_min - prev_x_min < 50 and
            keras_y_min - next_y_min < 10 and
            not (any(char.isdigit() for char in previous_prediction[0]) and any(not char.isdigit() for char in previous_prediction[0]))):

            # Combine predictions and update previous prediction info
            combined_text = previous_prediction[0] + prediction[0]
            combined_bbox = previous_prediction[1]
            list_bbox_forCombinedBbox.append(previous_prediction[1])
            list_bbox_forCombinedBbox.append(prediction[1])
            # Create a new combined prediction and append it to the list
            combined_prediction = (combined_text, combined_bbox)
            keras_ocr_results_list.append(combined_prediction)

        elif any(char.isdigit() for char in prediction[0]) and any(not char.isdigit() for char in prediction[0]):
            # If it contains a mix, do not combine and add it directly
            # keras_ocr_results_list.append(prediction)
            continue
        elif (i < len(prediction_group) - 1 and
              next_x_min - keras_x_min < 50 and
              keras_y_min - next_y_min < 10):

            if not (any(char.isdigit() for char in next_prediction[0]) and any(not char.isdigit() for char in next_prediction[0])):
                # Combine predictions with the next prediction
                combined_text = prediction[0] + next_prediction[0]
                combined_bbox = prediction[1]
                list_bbox_forCombinedBbox.append(prediction[1])
                list_bbox_forCombinedBbox.append(next_prediction[1])
                # Create a new combined prediction and append it to the list
                combined_prediction = (combined_text, combined_bbox)
                keras_ocr_results_list.append(combined_prediction)

        else:
            keras_ocr_results_list.append(prediction)

        # Store the current prediction info for the next iteration
        keras_x_min_precedente = keras_x_min
        keras_y_min_precedente = keras_y_min

        # Append current point and prediction to their respective lists

# Convert NumPy arrays to tuples
data_list_tuples = [(item[0], tuple(map(tuple, item[1]))) for item in keras_ocr_results_list]

# Create a set to eliminate duplicates, then convert back to the original format
unique_data_list_tuples = set(data_list_tuples)
keras_ocr_results_list = [(item[0], np.array(item[1], dtype=np.float32)) for item in unique_data_list_tuples]

list_keras_ocr_result = []

temp_list_keras_ocr_result = []

for prediction in keras_ocr_results_list:
    if prediction[0].isalpha() or len(prediction[0]) < 2:
        continue  # Ignore and skip unwanted predictions
    else:
        temp_list_keras_ocr_result.append(prediction)
#print(temp_list_keras_ocr_result)
keras_ocr_results_list = temp_list_keras_ocr_result

keras_point_list = []
for prediction in keras_ocr_results_list:
    keras_x_min, keras_y_min = prediction[1][0]
    point = keras_x_min, keras_y_min
    keras_point_list.append(point)

print(keras_ocr_results_list)
print(keras_point_list)

Looking for /root/.keras-ocr/craft_mlt_25k.h5
Looking for /root/.keras-ocr/crnn_kurapan.h5
1/1 [==============================] - 5s 5s/step
[[('r', array([[         62,           0],
       [         83,           0],
       [         83,          20],
       [         62,          20]], dtype=float32)), ('r', array([[        198,           0],
       [        220,           0],
       [        220,          22],
       [        198,          22]], dtype=float32)), ('r', array([[        333,           0],
       [        353,           0],
       [        353,          21],
       [        333,          21]], dtype=float32)), ('w', array([[     60.471,      30.598],
       [     132.75,      28.266],
       [      133.6,       54.69],
       [     61.323,      57.022]], dtype=float32)), ('w', array([[        195,          30],
       [        247,          30],
       [        247,          55],
       [        195,          55]], dtype=float32)), ('w', array([[        336,          3

In [ ]:
## connecting to the google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Pip install (recommended)

!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 614.2/614.2 kB 5.7 MB/s eta 0:00:00


In [ ]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.163 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.4/107.7 GB disk)


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO('/content/drive/MyDrive/technicalDrawingAnalysis/electronicsDrawing/training_results/electronicsComponents/weights/best0.pt')  # pretrained YOLOv8n model

# Run batched inference on a list of images
results = model([path_img])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes.data  # Boxes object for bbox outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    print(boxes)


0: 352x640 3 Resistances, 1507.7ms
Speed: 3.0ms preprocess, 1507.7ms inference, 29.2ms postprocess per image at shape (1, 3, 352, 640)


tensor([[316.8976,  25.6822, 380.8904,  58.2948,   0.8422,   3.0000],
        [180.0659,  25.1818, 247.5618,  58.4271,   0.8406,   3.0000],
        [ 46.1427,  24.9507, 113.3729,  58.1779,   0.8263,   3.0000]])


In [ ]:

yolo_result_list_before_Accuracy_test = []
yolo_result_list = []

# Convert tensor to list of lists using .tolist()
yolo_result_list_before_Accuracy_test = boxes.tolist()

# Check if confidence score is greater than 0.6 and append to yolo_result
for box in yolo_result_list_before_Accuracy_test:
    if box[-2] > 0.5:
        yolo_result_list.append(box)


# Print the converted list of lists
print(yolo_result_list)


[[316.8975524902344, 25.682239532470703, 380.8903503417969, 58.294795989990234, 0.8421622514724731, 3.0], [180.06590270996094, 25.181798934936523, 247.5618438720703, 58.42710876464844, 0.8405672311782837, 3.0], [46.14271926879883, 24.95070457458496, 113.37285614013672, 58.17792892456055, 0.8263255953788757, 3.0]]


In [ ]:
# Create an empty list to store tuples of first and second elements
yolo_point_list = []

# Iterate through each bbox in yolo_result_list
for box in yolo_result_list:
    # Extract the first and second elements and create a tuple
    center_tuple = (box[0], box[1])
    yolo_point_list.append(center_tuple)

# Print the list of tuples
print(yolo_point_list)

[(316.8975524902344, 25.682239532470703), (180.06590270996094, 25.181798934936523), (46.14271926879883, 24.95070457458496)]


In [ ]:
print(yolo_result_list)
print(list_keras_ocr_result)

[[50.15874099731445, 220.6107940673828, 105.17615509033203, 294.9666442871094, 0.8959755897521973, 4.0], [257.4380187988281, 247.91587829589844, 294.2219543457031, 337.0470886230469, 0.8478724956512451, 3.0], [128.9705047607422, 49.75953674316406, 208.2041015625, 88.89810180664062, 0.8208956718444824, 3.0], [255.34552001953125, 149.25747680664062, 297.29833984375, 188.6799774169922, 0.6954174637794495, 0.0]]
[]


In [ ]:

import numpy as np

# Create an empty NumPy array to store distances
distances = np.zeros((len(yolo_point_list), len(keras_point_list)))

# Calculate distances and store them in the array
for i, point1 in enumerate(yolo_point_list):
    x1, y1 = point1
    for j, point2 in enumerate(keras_point_list):
        x2, y2 = point2
        distance = np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
        distances[i, j] = distance

distances_matrix = np.copy(distances)

# Initialize a list to store the closest pairs as tuples (i, j)
closest_pairs_list = []

# Find the closest pairs and add them to the list
for _ in range(len(yolo_point_list)):
    min_distance_index = np.unravel_index(np.argmin(distances), distances.shape)
    i, j = min_distance_index
    closest_pairs_list.append((i, j))
    distances[i, :] = np.inf  # Set row i to infinity to exclude it from future comparisons
    distances[:, j] = np.inf  # Set column j to infinity to exclude it from future comparisons

print(closest_pairs_list)

# Initialize a dictionary to store the minimum distance for each j
min_distance_dict = {}

# Iterate through the closest_pairs_list to find the minimum distances
for i, j in closest_pairs_list:
    distance = distances_matrix[i, j]
    if j not in min_distance_dict or distance < min_distance_dict[j]:
        min_distance_dict[j] = distance

# Filter the closest_pairs_list based on the minimum distances
filtered_closest_pairs_list = [(i, j) for i, j in closest_pairs_list if distances_matrix[i, j] == min_distance_dict[j]]

# Remove duplicates from filtered_closest_pairs_list
filtered_closest_pairs_list = list(set(filtered_closest_pairs_list))
print(filtered_closest_pairs_list)



[(0, 2), (1, 1), (2, 0)]
[(1, 1), (0, 2), (2, 0)]


In [ ]:
import re

indexes = filtered_closest_pairs_list

for yolo_index, keras_index in indexes:
    x1, y1, x2, y2, accuracy, yolo_class = yolo_result_list[yolo_index]
    keras_text = keras_ocr_results_list[keras_index][0]
    keras_bbox = keras_ocr_results_list[keras_index][1]

    if keras_text[0].isalpha():
      # Separate digits from non-digits
      digits = ''.join(re.findall(r'\d', keras_text))
      non_digits = ''.join(re.findall(r'\D', keras_text))

      # Reverse the non-digit part
      keras_text = digits + non_digits[::]

    for i in keras_text:
         if i == 'u' :
           keras_text = keras_text[:-2] + 'µ' + keras_text[-1:]

    if yolo_class == 4.0:
      keras_text = keras_text[:-1] + 'V'

    if yolo_class == 3.0:
      keras_text = keras_text[:-1] + 'Ω'

    print("Class:", yolo_class)
    print("YOLO BBox : (", x1, ",", y1, ",",  x2, ",", y2,")")
    print("Keras OCR Text:", keras_text)
    print("Keras OCR BBox:", keras_bbox)
    print("---")

Class: 3.0
YOLO BBox : ( 180.06590270996094 , 25.181798934936523 , 247.5618438720703 , 58.42710876464844 )
Keras OCR Text: 300Ω
Keras OCR BBox: [[        174          63]
 [        228          63]
 [        228          85]
 [        174          85]]
---
Class: 3.0
YOLO BBox : ( 316.8975524902344 , 25.682239532470703 , 380.8903503417969 , 58.294795989990234 )
Keras OCR Text: 50Ω
Keras OCR BBox: [[        315          63]
 [        374          63]
 [        374          85]
 [        315          85]]
---
Class: 3.0
YOLO BBox : ( 46.14271926879883 , 24.95070457458496 , 113.37285614013672 , 58.17792892456055 )
Keras OCR Text: 1ooΩ
Keras OCR BBox: [[     34.549      63.422]
 [     106.84      61.568]
 [     107.44      85.168]
 [     35.154      87.022]]
---
